# Preparing Colab

In [1]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving configure.py to configure.py
Saving data.zip to data.zip
Saving data_collector.py to data_collector.py
Saving model.py to model.py
Saving predict_from_chkpnt.py to predict_from_chkpnt.py
Saving run_model.py to run_model.py
Saving saved_model.zip to saved_model.zip
Saving utility.py to utility.py


In [2]:
from zipfile import ZipFile
with ZipFile('data.zip', 'r') as zip:
  zip.extractall()
  print('Done data')
with ZipFile('saved_model.zip', 'r') as zip:
  zip.extractall()
  print('Done saved_model')

Done data
Done saved_model


# RAPTA Demo Version
This demo version is for tutorial purpose. Here we analyzed only one file of PT_S38417 for 0.78V.

In [3]:
# Python Libraries
import time
import pickle
import numpy as np

# Repositories
All the necessary codes have been developed in different files.

In [4]:
# Customized Libraries
import configure as config # For model configuration
import data_collector # For data processing
import model # Model has been developed here
import run_model # Model training related code
import predict_from_chkpnt # Testing related code
import utility # Any other necessary functions for this project

Running on the CPU
Running on the CPU


## Data Generation and Preparation:
We used Synopsys IC Compiler (ICC) for the physical design of benchmarks in 32nm technology available under Synopsys educational license. We used the ICC timing report to extract a set of timing paths and queried ICC to extract the gate and net properties. We used one-hot encoding to represent the value of non-numerical gate properties, such as "gate type” or "threshold voltage”. Synopsys Primetime was then launched for path-based timing analysis, and the delay of each subpath (𝑃𝐷 , 𝑃𝐶, and 𝑃𝐿) was collected as labels and sub-labels for each timing path, respectively.

Raw data is in json format. We formatted and converted the data in pickle format. The necessary functions are in the "data_collector.py" file. So, we import that here at first.

In [6]:
fp=config.fp[0:-1]
raw_input_file = config.processed_saved_path + fp + '/jason.json'
processed_saved_path = config.processed_saved_path + fp + '/'
processed_file = config.processed_saved_path + fp + '/processed.json'
print(' starting file processing {}'.format(fp))
file_processing_tic = time.perf_counter()
data_collector.file_processing(raw_input_file, processed_file)
file_processing_toc = time.perf_counter()
print('file_processed {}'.format(fp))
data_collector.preprocess_data(processed_file, processed_saved_path, fp)
preprocess_data_toc = time.perf_counter()
print('preprocess_data {}, file processing time {}, preprocessing_data {}'.format(fp, (
            file_processing_toc - file_processing_tic), (preprocess_data_toc - file_processing_tic)))
X = pickle.load(open(config.exp_data_path + fp + '_X_gate.pk', 'rb'))
X_sublabel = pickle.load(open(config.exp_data_path + fp + '_X_sublabels.pk', 'rb'))
Y = pickle.load(open(config.exp_data_path + fp + '_Y_gate.pk', 'rb'))
saving_Path_Train = config.exp_data_path + fp + '_train.pk'
saving_Path_Test = config.exp_data_path + fp + '_test.pk'
print(' starting file processing {}'.format(fp))
data_collector.createFromProcessed(X, X_sublabel, Y, saving_Path_Train, saving_Path_Test)
print('preprocess_data {}'.format(fp))

 starting file processing v_0.78
file_processed v_0.78
preprocess_data v_0.78, file processing time 0.4244836190000001, preprocessing_data 3.5743583660000127
 starting file processing v_0.78


/content/data_collector.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_arr = np.array(X)
/content/data_collector.py:132: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_new = np.array(X_new) # l, d, c, VS, sublabel, label


preprocess_data v_0.78


## Model Training

In [7]:
# Training Model
print(config.X_train_path_single)
X_train_org = pickle.load(open(config.X_train_path_single, 'rb'))
X_train, X_valid = run_model.train_test_split(X_train_org, test_size=0.2, random_state=42, shuffle=True)  # Split the set
print(len(X_train))
# running model
tic = time.perf_counter()
run_model.main(np.array(X_train), np.array(X_valid), config.fp)
toc = time.perf_counter()
print("Model took {} seconds".format((toc - tic)))

data/model_purpose/PT_S38417/v_0.78_train.pk
3324
Running Main for v_0.78_
Running tensormaker
Train Input Data Fit Transformed
Train Target Data Fit Transformed
Running tensormaker
Test Input Data Transformed
Test Target Data Transformed
Iteration: 0/10  Current Loss: 0.6858076453208923 and previously recorded Min Loss None
Iteration: 0/20  Current Loss: 0.26422178745269775 and previously recorded Min Loss 0.6858076453208923
 Model saved
Iteration: 1/30  Current Loss: 0.08295884728431702 and previously recorded Min Loss 0.26422178745269775
 Model saved
Iteration: 1/40  Current Loss: 0.055782463401556015 and previously recorded Min Loss 0.08295884728431702
 Model saved
Iteration: 1/50  Current Loss: 0.0260595865547657 and previously recorded Min Loss 0.055782463401556015
 Model saved
Iteration: 2/60  Current Loss: 0.02114274352788925 and previously recorded Min Loss 0.0260595865547657
 Model saved
Iteration: 2/70  Current Loss: 0.01573525369167328 and previously recorded Min Loss 0.021

## Model Testing

In [8]:
# Testing Model
combined_setup_list = [config.fp]
exp_name_list = [config.fp]
for i in range(len(exp_name_list)):
    print('Running for {}'.format(exp_name_list[i]))
    chkpoint_filename = config.model_chkpnt_path + combined_setup_list[i] + 'model_min_loss.ckpt'
    std_scalar_filename = config.model_chkpnt_path + combined_setup_list[i] + 'std_scaler_train.pk'
    std_y_filename = config.model_chkpnt_path + combined_setup_list[i] + 'sc_y_train.pk'
    training_time_filename = config.model_chkpnt_path + combined_setup_list[i] + 'training_time.pk'
    exp_name=exp_name_list[i]
    predict_from_chkpnt.predict_from_chkpnt(chkpoint_filename, std_scalar_filename, std_y_filename, exp_name, training_time_filename)


Running for v_0.78_
Model loaded successfully
Running tensormaker
Test Input Data Transformed
Test Target Data Transformed
Starting files set v_0.78_
X file data/model_purpose/PT_S38417/v_0.78_test.pk
 Model saved
Model from v_0.78_, Test data v_0.78_, Loss: 0.00039662918425165117 


## Performance Metrics

In [9]:
# Metrics
# reading processed data
labels = pickle.load(open(config.label_file, 'rb')).flatten()
outputs = pickle.load(open(config.output_file, 'rb')).flatten()
errors = labels - outputs
sigma = np.std(errors) * 1000  # ns to ps
mu = np.ndarray.mean(errors) * 1000  # ns to ps
errors_percentage = (np.std(errors) / max(labels)) * 100
mse = np.square(np.subtract(labels,outputs)).mean()
print('Std error {}, mean error {}, mse {}, errors_percentage {}'.format(sigma, mu, mse, errors_percentage))
filename = config.model_chkpnt_path+config.fp
utility.save_as_csv(labels, outputs, errors, filename)

Std error 10.302479950052989, mean error -0.7800114884800924, mse 0.0001067495110434048, errors_percentage 0.9937716081289266
